In [51]:
import pandas as pd
import numpy as np
import numpy as np
import re
import nltk
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from itertools import combinations
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
df = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz', lines=True)
df = df.iloc[0:100]

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    word_tokens = word_tokenize(text)
    text = [w for w in word_tokens if not w in stop_words]
    text = list(dict.fromkeys(text))
    return text

df['reviews'] = df['reviewText'].apply(clean_text)

In [0]:
words = []
for x in range (df.shape[0]):
  words += df['reviews'][x]

words = list(dict.fromkeys(words))

In [55]:
combs = list(combinations(words,2))
len(combs)

2278045

In [0]:
corpus = []
for x in range (df.shape[0]):
  corpus.append(df['reviews'][x])

In [0]:
model = Word2Vec(corpus, size=100, window=5, min_count=1, workers=4)

In [58]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

bought
first
hpc
served
faithfully
lost
travelling
searched
another
one


In [59]:
sim_scores = []
for i in range(len(combs)):
  sim_scores.append(model.similarity(combs[i][0], combs[i][1]))
len(sim_scores)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


2278045

In [0]:
Combination_Similarities = pd.DataFrame(list(zip(combs, sim_scores)), columns=['Combinations', 'Similarity_score'])

In [47]:
Combination_Similarities.head()

,Combinations,Similarity_score
0,"(bought, first)",-0.064318
1,"(bought, hpc)",0.026792
2,"(bought, served)",-0.010006
3,"(bought, faithfully)",-0.039100
4,"(bought, lost)",-0.101387
